In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
 from tqdm.notebook import tqdm
tqdm.pandas()
 import os
os.getcwd()
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import re

In [ ]:
#read csv as df
df = pd.read_csv('/Users/dimabuhanevyc/Documents/OpenMinds/test_assignment_data.csv')

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0  

 def detect_language(text):
    try:
        return detect(text)
    except:
        return "error"  

 df['language'] = df['fullText'].progress_apply(detect_language)
df['language'].value_counts()

In [ ]:
from datetime import date
today = date.today() 

In [ ]:
date.fromisoformat('2019-12-04')

In [ ]:
df['Date'] = pd.to_datetime(df['pubTime']).dt.date
df.Date.min()
df.Date.max()

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("ru_core_news_lg")


 stop_words = nlp.Defaults.stop_words

 def preprocess_text(text):
    doc = nlp(text)
     preprocessed_text = ' '.join([token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space and token.text.lower() not in stop_words])
    return preprocessed_text

In [ ]:
df['processed_text'] = df['fullText'].progress_apply(preprocess_text)

 import os
 os.getcwd()

 df.to_json('om_test_processedtext.json', orient='records', lines=True)

 df = pd.read_json('om_test_processedtext.json', lines=True)

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [ ]:
model_name = "DeepPavlov/rubert-base-cased"   
model = SentenceTransformer(model_name)

In [ ]:
embeddings = []
for text in tqdm(df['processed_text'].tolist(), desc="Generating embeddings"):
    embedding = model.encode(text, show_progress_bar=False)
    embeddings.append(embedding)

In [ ]:
#save embeddings as json
embeddings_list = [embedding.tolist() for embedding in embeddings]

 with open('embeddings.json', 'w') as f:
    json.dump(embeddings_list, f)

In [ ]:
import numpy as np
import json

 with open('embeddings.json', 'r') as f:
    embeddings_list = json.load(f)

 embeddings = np.array(embeddings_list)

In [ ]:
date_min = df.Date.min()
date_max = df.Date.max()

In [ ]:
# Visualize the topics
# Initialize and fit BERTopic
topic_model = BERTopic(language=None, calculate_probabilities=True)

In [ ]:
docs = df['processed_text'].tolist()

In [ ]:
topics, probs = topic_model.fit_transform(docs)

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
fig = topic_model.visualize_hierarchy()
fig.update_layout(width=600, height=1000)  
fig.show()

In [ ]:
tree = topic_model.get_topic_tree(hierarchical_topics)

In [ ]:
print(tree)

 topic_model.visualize_barchart(top_n_topics=11, n_words=10)

In [ ]:
topic_info = topic_model.get_topic_info()

 for index, row in topic_info.iterrows():
    if row['Topic'] != -1:  # -1 is noise
        topic_num = row['Topic']
        print(f"\nTopic: {topic_num}")
        print(f"Size: {row['Count']}")

         words_scores = topic_model.get_topic(topic_num)  # c-TF-IDF scores

         top_n_words = words_scores[:10]  
        top_words = [word for word, score in top_n_words]
        print(f"Top words: {top_words}")



 largest_topics = topic_info.sort_values('Count', ascending=False)

##

In [ ]:
from umap import UMAP
import hdbscan
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import textwrap

In [ ]:
date_min_formatted = date_min.strftime('%A, %b, %d')
date_max_formatted = date_max.strftime('%A, %b, %d')

In [ ]:
umap_model_2d = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42)
reduced_embeddings_2d = umap_model_2d.fit_transform(embeddings)

In [ ]:
df['x_coordinate'] = reduced_embeddings_2d[:, 0]
df['y_coordinate'] = reduced_embeddings_2d[:, 1]

In [ ]:
df['cluster_labels_2d'] = cluster_labels_2d

In [ ]:
n_clusters_2d = len(set(cluster_labels_2d)) - (1 if -1 in cluster_labels_2d else 0)
n_noise_2d = list(cluster_labels_2d).count(-1)

In [ ]:
print(f"Number of clusters found: {n_clusters_2d}")
print(f"Number of noise points: {n_noise_2d}")

In [ ]:
df['cluster_count'] = df.groupby('cluster_labels_2d')['cluster_labels_2d'].transform('count')

In [ ]:
def wrap_text(text):
    return "<br>".join(textwrap.wrap(text, width=30))

In [ ]:
#replace <br> with \n
df['fullText'] = df['fullText'].str.replace('<br>', '')
# Generate hover text with cluster information, full text, and post counts
hover_texts_2d = df.apply(lambda row: f"Cluster: {row['cluster_labels_2d']}<br>Posts in Cluster: {row['cluster_count']}<br>Full Text: {wrap_text(row['fullText'])}", axis=1)

In [ ]:
# Create the 2D scatter plot
fig_2d = px.scatter(
    df, x=reduced_embeddings_2d[:, 0], y=reduced_embeddings_2d[:, 1],
    color="cluster_labels_2d",  # Ensure this uses the updated 2D cluster labels
    hover_data=[hover_texts_2d],
    title=f"<b>Topic Modeling and Clustering Analysis</b><br>Between {date_min_formatted} and {date_max_formatted}",
    labels={"hover_data_0": "Details"}  # Rename hover data label
)

In [ ]:
# Update the layout to include an annotation for clusters and Greek values, and remove legend
fig_2d.update_layout(
    annotations=[{
        'text': f"Clusters (Κ): {n_clusters_2d}<br>Noise Points (η): {n_noise_2d}<br>Clustering Params: Min Cluster Size (Ν) = 20, Min Samples (Σ) = 1",
        'showarrow': False,
        'x': 1,
        'xref': "paper",
        'xanchor': 'right',
        'xshift': 0,
        'y': 1,
        'yref': "paper",
        'yanchor': 'top',
        'align': "right",
        'font': {
            'size': 10  # Make font even smaller
        }
    }],
    showlegend=False
)
#delete colorbar
fig_2d.update_coloraxes(colorbar_title=None)
#make markers smaller
fig_2d.update_traces(marker=dict(size=3))
#make background white
fig_2d.update_layout(plot_bgcolor='white')
#make bigger
fig_2d.update_layout(width=800, height=800)

In [ ]:
#remove colorbar
fig_2d.update_layout(coloraxis_showscale=False)
# Display the figure
fig_2d.show()

In [ ]:
# Save your Plotly figure as an HTML file
fig_2d.write_html("/Users/dimabuhanevyc/Documents/Research_Analysis/RandomResearch/openminds/visualization.html", include_plotlyjs='cdn')

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter

In [ ]:
# 1. Group the DataFrame by cluster labels
clustered_groups = df.groupby('cluster_labels_2d')

In [ ]:
# Initialize lists to store central text and top words per cluster
central_texts = []
top_words_per_cluster = []
# Iterate over each cluster
for cluster_label, cluster_data in clustered_groups:
    # Calculate centroid of the cluster
    centroid = cluster_data[['x_coordinate', 'y_coordinate']].mean(axis=0).values.reshape(1, -1)
    
    # Calculate Euclidean distances between each point in the cluster and the centroid
    distances = euclidean_distances(cluster_data[['x_coordinate', 'y_coordinate']], centroid)
    
    # Find the index of the point with the smallest distance (i.e., the central text)
    central_text_index = distances.argmin()
    
    # Append the central text to the list
    central_texts.append(cluster_data.iloc[central_text_index]['fullText'])
    
    # Get all text data in the cluster
    texts_in_cluster = cluster_data['processed_text']  # Using processed_text for counting top words
    
    # Tokenize and count words in the cluster
    all_words = ' '.join(texts_in_cluster).split()
    word_counts = Counter(all_words)
    
    # Select the top 10 words based on their frequency
    top_words = word_counts.most_common(10)
    
    # Append the top words to the list
    top_words_per_cluster.append(top_words)

In [ ]:
# Print central texts and top words per cluster
for i, (central_text, top_words) in enumerate(zip(central_texts, top_words_per_cluster)):
    print(f"Cluster {i}:")
    print("Central Text:", central_text)
    print("Top 10 Words:", top_words)
    print()

In [ ]:
import plotly.express as px

Calculate top words and central texts per cluster (assuming you've already run the code for this)
Code for calculating top words and central texts per cluster goes here...

In [ ]:
# Generate hover text with cluster information, central text, top 10 words, and post counts
hover_texts_2d = df.apply(lambda row: f"Cluster: {row['cluster_labels_2d']}<br>Top 10 Words: {' '.join([word for word, _ in top_words_per_cluster[row['cluster_labels_2d']]])}<br>Central Text: {wrap_text(central_texts[row['cluster_labels_2d']])}", axis=1)

In [ ]:
# Create the 2D scatter plot
fig_2d = px.scatter(
    df, x=reduced_embeddings_2d[:, 0], y=reduced_embeddings_2d[:, 1],
    color="cluster_labels_2d",  # Ensure this uses the updated 2D cluster labels
    hover_data=[hover_texts_2d],
    title=f"<b>Topic Modeling and Clustering Analysis</b><br>Between {date_min_formatted} and {date_max_formatted}",
    labels={"hover_data_0": "Details"}  # Rename hover data label
)

In [ ]:
# Update the layout to include an annotation for clusters and Greek values, and remove legend
fig_2d.update_layout(
    annotations=[{
        'text': f"Clusters (Κ): {n_clusters_2d}<br>Noise Points (η): {n_noise_2d}<br>Clustering Params: Min Cluster Size (Ν) = 20, Min Samples (Σ) = 1",
        'showarrow': False,
        'x': 1,
        'xref': "paper",
        'xanchor': 'right',
        'xshift': 0,
        'y': 1,
        'yref': "paper",
        'yanchor': 'top',
        'align': "right",
        'font': {
            'size': 10  # Make font even smaller
        }
    }],
    showlegend=False
)

In [ ]:
# Update hover mode and add custom formatting
fig_2d.update_traces(hovertemplate='%{customdata[0]}')

In [ ]:
# Delete colorbar
fig_2d.update_coloraxes(colorbar_title=None)

In [ ]:
# Make markers smaller
fig_2d.update_traces(marker=dict(size=3))

In [ ]:
# Make background white
fig_2d.update_layout(plot_bgcolor='white')

In [ ]:
# Make figure bigger
fig_2d.update_layout(width=800, height=800)

In [ ]:
# Remove colorbar
fig_2d.update_layout(coloraxis_showscale=False)

In [ ]:
# Display the figure
fig_2d.show()

printing main clusters by count, excluding noise

In [ ]:
cluster_sizes = df[df['cluster_labels_2d'] != -1].groupby('cluster_labels_2d').size().sort_values(ascending=False)

In [ ]:
# Select top 10 clusters
top_10_clusters = cluster_sizes.head(10).index.tolist()

Assuming `date_min_formatted` and `date_max_formatted` are defined as before
Assuming `df` is your DataFrame and `cluster_labels_2d` is the column with cluster labels

In [ ]:
# First, let's get the overall context and introduction
print(f"Between {date_min_formatted} and {date_max_formatted}, we have analyzed {len(df)} texts and identified {n_clusters_2d} clusters (excluding noise).")

In [ ]:
# Now, let's mention the summary about the top 10 clusters and their central texts
print("The top 10 clusters and their central texts are below:")

Assuming `cluster_sizes` and `top_10_clusters` are defined as before
Assuming `central_texts` is a dictionary or list indexed by cluster label as appropriate

In [ ]:
for cluster_label in top_10_clusters:
    # Get the cluster size
    cluster_size = cluster_sizes[cluster_label]
    
    # Find the central text for this cluster
    central_text = central_texts[cluster_label]  # Make sure central_texts is indexed correctly
    
    print(f"Cluster Number: {cluster_label}, Count: {cluster_size}")
    print(f"Central Text: {central_text}")
    print("-----")

In [ ]:
# Mentioning the graph
print("The graph showing the total distribution of texts in our model is below.")

In [ ]:
# Display the graph
# Assuming `fig_2d` is your Plotly figure for the graph
fig_2d.show()